## Ejercicio 12

Utilice una red neuronal formada por una única capa de salida de 3 neuronas para clasificar las muestras de vino del archivo Vinos.csv descripto en el ejercicio anterior.

Realice 30 ejecuciones independientes utilizando el 60% y 80% de los ejemplos como entrenamiento y el resto como testeo.

Utilice un máximo de 400 iteraciones y velocidades de aprendizaje 0.1, 0.2 y 0.3.

Complete la siguiente tabla con los resultados de las siguientes configuraciones


In [5]:
import numpy as np
import pandas as pd
from Fuentes.ClassRNMulticlase import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

MAX_ITER = 400
COTA = 1e-03

DATOS_DIR   = '../../Datos/'
data = pd.read_csv(DATOS_DIR + 'vinos.csv', sep=';')

T = pd.get_dummies(data["Class"]).astype(int).to_numpy()
X = data.iloc[:, 1:]

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

def run_experiments(X, T, FUN="sigmoid", COSTO="ECM", 
                    max_iter=400, cota=1e-3,
                    train_sizes=[0.6, 0.8],
                    alphas=[0.1, 0.2, 0.3],
                    n_runs=30):
    
    resultados = []
    
    for perc_for_training in train_sizes:
        for alpha in alphas:
            acc_train_runs, acc_test_runs, iters_runs = [], [], []

            for _ in range(n_runs):
                X_train, X_test, T_train, T_test = train_test_split(
                    X, T, train_size=perc_for_training, stratify=T, random_state=None
                )
                
                nnp = RNMulticlase(
                    alpha=alpha, n_iter=max_iter, cotaE=cota, FUN=FUN, COSTO=COSTO
                )
                nnp.fit(X_train, T_train)
                
                acc_train = nnp.accuracy(X_train, T_train)
                acc_test  = nnp.accuracy(X_test, T_test)
                
                acc_train_runs.append(acc_train)
                acc_test_runs.append(acc_test)
            
            resultados.append({
                "train_size": perc_for_training,
                "alpha": alpha,
                "mean_acc_train": np.mean(acc_train_runs),
                "mean_acc_test": np.mean(acc_test_runs),
            })
        
        df_temp = pd.DataFrame([r for r in resultados if r["train_size"] == perc_for_training])
        resultados.append({
            "train_size": perc_for_training,
            "alpha": "general",
            "mean_acc_train": df_temp["mean_acc_train"].mean(),
            "mean_acc_test": df_temp["mean_acc_test"].mean(),
        })
    
    return pd.DataFrame(resultados)

### a)

Función de activación ‘sigmoid’ y función de costo ‘ECM’ (error cuadrático medio)

In [6]:
run_experiments(X, T, FUN="sigmoid", COSTO="ECM", max_iter=MAX_ITER, cota=COTA)

,train_size,alpha,mean_acc_train,mean_acc_test
0,0.6,0.1,0.952201,0.926389
1,0.6,0.2,0.967610,0.936574
2,0.6,0.3,0.963208,0.947222
3,0.6,general,0.961006,0.936728
4,0.8,0.1,0.951174,0.932407
5,0.8,0.2,0.966197,0.926852
6,0.8,0.3,0.969718,0.943519
7,0.8,general,0.962363,0.934259


### b) 

Función de activación ‘sigmoid’ y función de costo ‘EC_binaria’ (entropía cruzada binaria)


In [7]:
run_experiments(X, T, FUN="sigmoid", COSTO="EC_binaria", max_iter=MAX_ITER, cota=COTA)

,train_size,alpha,mean_acc_train,mean_acc_test
0,0.6,0.1,0.971384,0.943519
1,0.6,0.2,0.973270,0.949537
2,0.6,0.3,0.972327,0.950926
3,0.6,general,0.972327,0.947994
4,0.8,0.1,0.971831,0.943519
5,0.8,0.2,0.973709,0.955556
6,0.8,0.3,0.971831,0.965741
7,0.8,general,0.972457,0.954938


### c) 

Función de activación ‘tanh’ y función de costo ‘ECM’ (error cuadrático medio)

In [8]:
T_tanh = 2 * T - 1
run_experiments(X, T_tanh, FUN="tanh", COSTO="ECM", max_iter=MAX_ITER, cota=COTA)

,train_size,alpha,mean_acc_train,mean_acc_test
0,0.6,0.1,0.980503,0.954630
1,0.6,0.2,0.981132,0.941667
2,0.6,0.3,0.977673,0.938889
3,0.6,general,0.979769,0.945062
4,0.8,0.1,0.980282,0.948148
5,0.8,0.2,0.975822,0.954630
6,0.8,0.3,0.970892,0.938889
7,0.8,general,0.975665,0.947222


### d) 

Capa ‘Softmax’ y función de costo ‘EC’ (entropía cruzada)

In [ ]:
run_experiments(X, T, FUN="softmax", COSTO="EC", max_iter=MAX_ITER, cota=COTA)

,train_size,alpha,mean_acc_train,mean_acc_test
0,0.6,0.1,0.987736,0.937963
1,0.6,0.2,0.986792,0.936111
2,0.6,0.3,0.982390,0.923148
3,0.6,general,0.985639,0.932407
4,0.8,0.1,0.987089,0.944444
5,0.8,0.2,0.983333,0.935185
6,0.8,0.3,0.976761,0.949074
7,0.8,general,0.982394,0.942901
